In [1]:
import numpy as np
import pandas as pd
import os
# from matplotlib import pyplot as plt

# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.api import ARIMA
import cupy as cp
from cuml.tsa.arima import ARIMA
# from statsmodels.tsa.api import VAR
# import statsmodels.api as sm
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

import torch
from torchmetrics.functional import classification, accuracy, auroc, f1_score, confusion_matrix
import torch.nn.functional as F
from tqdm import tqdm


import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())


/data/wchao/envs/rapids-23.06/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of processors:  48


# Read Data

In [2]:
data_dir = "/data/wchao/data/skill_inflow_outflow/"
dataset = "cons"
subgraph_num=7446
subgraph_num = subgraph_num
startdate = pd.Timestamp(2017,10,1)
enddate = pd.Timestamp(2019,4,1)
period = 'M'
time_attr = pd.date_range(start=startdate, end=enddate, freq=period).to_period(period)
time_attr = time_attr.to_series().astype(str)
datasetinflow =  pd.read_csv(
    os.path.join(data_dir, f"skill_inflow_list_{dataset}.csv"),
    encoding='utf-8',
    header=0,
    on_bad_lines='warn',
)
datasetinflow.index = time_attr
datasetinflow.drop("time_attr", axis=1, inplace=True, errors="ignore")
datasetoutflow =  pd.read_csv(
    os.path.join(data_dir, f"skill_outflow_list_{dataset}.csv"),
    encoding='utf-8',
    header=0,
    on_bad_lines='warn'
)
datasetoutflow.index = time_attr
datasetoutflow.drop("time_attr", axis=1, inplace=True, errors="ignore")
datasetinflow = datasetinflow.iloc[:,:subgraph_num]
datasetoutflow = datasetoutflow.iloc[:,:subgraph_num]
time_range = datasetinflow.columns.value_counts().sort_index().index.tolist()


# Preprocessing

In [3]:
def _get_matrices(count, time_range):
    '''get normalized company-position pairwise matrix data
    '''
    matrices = {}
    # build matrix
    for time in time_range:
        matrix = count[time].astype('float32')
        matrix[matrix.isna()] = 0.0
        matrix += 1e-6  # avoid divided by 0
        matrices[time] = torch.from_numpy(matrix.values)
    # stack data
    matrices = torch.stack(list(matrices.values()),dim=0).float()
    # print(torch.std(matrices,dim=0,unbiased=True).shape)
    # print((torch.std(matrices,dim=0,unbiased=True)==0).any())
    # normalize data
    print(matrices.shape)
    matrices = F.normalize(matrices, dim=1)
    # print(matrices.shape)
    # print(matrices)
    # matrices_submean = torch.sub(matrices, torch.mean(matrices, dim=0))
    # matrices = torch.div(matrices_submean, torch.maximum(torch.std(matrices,dim=0,unbiased=True), torch.tensor(1)))
    return matrices

inflow_matrices = _get_matrices(datasetinflow, time_range)
outflow_matrices = _get_matrices(datasetoutflow, time_range)

torch.Size([7446, 18])
torch.Size([7446, 18])


torch.Size([7446, 18])
torch.Size([7446, 18])


# Label

In [4]:
def _get_labels(vec: torch.Tensor, class_num: int):
    '''split all range for data and get value range for each labels
    '''
    vec_tmp: torch.Tensor = vec.reshape(-1)
    vec_tmp, _ = vec_tmp.sort()
    n = len(vec_tmp)
    val_range_list: list = []
    for i in range(class_num):
        val_range_list.append(vec_tmp[(n // class_num) * i])
    val_range_list.append(vec_tmp[-1])
    return val_range_list

# def _get_label_SAX(vec: torch.Tensor, class_num: int):
    
def _to_label(vec: torch.Tensor, val_range_list: list, class_num: int):
    '''map continuous values to `class_num` discrete labels for `vec` using `val_range_list`
    '''

    def _to_label_(v: float, val_range_list: list, class_num: int):
        if v < val_range_list[0]:
            return 0
        for i in range(class_num):
            if val_range_list[i] <= v <= val_range_list[i + 1]:
                return i
        return class_num - 1

    return vec.clone().apply_(lambda x: _to_label_(x, val_range_list, class_num)).long()

In [5]:
data = []
val_range_list = []
class_num = 5
for i in range(1,3):
    supply_x = inflow_matrices[:,:-i]
    supply_y = inflow_matrices[:,inflow_matrices.shape[1]-i]
    demand_x = outflow_matrices[:,:-i]
    demand_y = outflow_matrices[:,outflow_matrices.shape[1]-i]
    sample = {'supply_x': supply_x, 'supply_y': supply_y, "demand_x": demand_x, "demand_y": demand_y}
    data.append(sample)

for sample in data:
    val_range_list_supply = _get_labels(sample["supply_y"], class_num)
    val_range_list_demand = _get_labels(sample["demand_y"], class_num)
    val_range_list.append({"supply": val_range_list_supply, "demand":val_range_list_demand})
    sample["supply_y_label"] = _to_label(sample["supply_y"], val_range_list_supply, class_num)
    sample["demand_y_label"] = _to_label(sample["demand_y"], val_range_list_demand, class_num)

# ARIMA

In [10]:
sample = data[0]
sample["supply_x"].numpy()[:10,:].shape

(10, 17)

In [10]:
model = ARIMA(sample["supply_x"].numpy()[:2,:], order=(5,1,2))
results = model.fit()
model.forecast(0)

In [56]:
sample = data[0]

torch.Size([7446, 17])

In [ ]:
sample = data[0]
sd = "supply"
length = 10
pred = []
for i in tqdm(range(length)):
    model = ARIMA(sample[sd+"_x"].numpy()[:i,:], order=(sample[sd+"_x"].shape[1],1,int(sample[sd+"_x"].shape[1]/2)))
    # model.initialize_approximate_diffuse()
    results = model.fit()
    pred.append(results.forecast()[0])
output = _to_label(torch.tensor(pred), val_range_list[0][sd], class_num)
output = F.one_hot(output)
sample[sd+"_y_output"] = output

In [ ]:
output = _to_label(torch.tensor(pred), val_range_list[0][sd], class_num)
output = F.one_hot(output)
sample[sd+"_y_output"] = output

In [12]:
sd = "supply"
np.random.normal(0,1e-6,sample[sd+"_x"].numpy().shape[0])

array([ 1.68275778e-06, -1.93285781e-06,  1.84868182e-06, ...,
        1.13720849e-06, -1.97905934e-06, -1.02878404e-06])

In [13]:
def arima_forecasting(sample, order):
    model = ARIMA(sample, order=order)
    model.initialize_approximate_diffuse()
    results = model.fit()
    return results.forecast()[0]

In [29]:
rand_num = np.random.normal(0,1e-6,sample[sd+"_x"].numpy().shape[1])
model = ARIMA(sample[sd+"_x"].numpy()[:100,:].T+rand_num[...,None], order=(5,1,3))
results = model.fit()
pred = results.forecast(1)
pred.shape

[W] [18:00:30.375568] fit: Some batch members had optimizer problems


(1, 100)

In [6]:
sample = data[0]

sup_or_dem = ["supply", "demand"]
for idx, sample in enumerate(data):
    for sd in sup_or_dem:
        pred = []
        rand_num = np.random.normal(0,1e-6,sample[sd+"_x"].numpy().shape[1])
        model = ARIMA(sample[sd+"_x"].numpy()[:,:].T+rand_num[...,None], order=(5,1,3))
        # model.initialize_approximate_diffuse()
        results = model.fit()
        pred = results.forecast(1)
        output = _to_label(torch.tensor(pred), val_range_list[idx][sd], class_num)
        output = F.one_hot(output)
        sample[sd+"_y_output"] = output

[W] [10:20:37.231987] fit: Some batch members had optimizer problems
[W] [10:22:40.574145] fit: Some batch members had optimizer problems
[W] [10:25:44.806278] fit: Some batch members had optimizer problems
[W] [10:27:39.245048] fit: Some batch members had optimizer problems


In [30]:
print(pred.shape)
print(val_range_list[idx][sd])
# output = _to_label(torch.tensor(pred), val_range_list[idx][sd], class_num)

(1, 100)
[tensor(0.0009), tensor(0.1545), tensor(0.2184), tensor(0.2568), tensor(0.3135), tensor(0.8568)]


# Evaluation

F1, Auc Accuracy

In [7]:
def accuracy_metric(output: torch.Tensor, labels: torch.Tensor, class_num=10):
    '''evaluate model in three classification metrics: accuracy, weighted f1 score and auroc
    '''
    pred = torch.argmax(output, dim=-1)
    # print(pred.shape, labels.shape)
    acc = classification.accuracy(pred, labels, task="multiclass", num_classes=class_num)
    # print(acc)
    return acc

def f1_metric(output: torch.Tensor, labels: torch.Tensor, class_num=10):
    pred = torch.argmax(output, dim=-1)
    weighted_f1 = f1_score(pred, labels, task="multiclass", average='weighted', num_classes=class_num)
    return weighted_f1

def aucroc_metric(output: torch.Tensor, labels: torch.Tensor, class_num=10):
    au = auroc(output, labels, task='multiclass', num_classes=class_num)
    return au

def top_k_acc(output, target, k=3):
    with torch.no_grad():
        pred = torch.topk(output, k, dim=1)[1]
        assert pred.shape[0] == len(target)
        correct = 0
        for i in range(k):
            correct += torch.sum(pred[:, i] == target).item()
    return correct / len(target)

In [8]:
sup_or_dem = ["supply", "demand"]
combined_output_list = []
combined_label_list = []
for sd in sup_or_dem:
    output_list = []
    label_list = []
    for idx, sample in enumerate(data):
        output_list.append(sample[sd+"_y_output"])
        label_list.append(sample[sd+"_y_label"])
    # print(output.shape)
    output = torch.cat(output_list,dim=1).squeeze().float()
    # print(output.shape)
    label = torch.cat(label_list,dim=0)
    # print(label.shape)
    print(sd+"_accuracy:", accuracy_metric(output,label,class_num).item())
    print(sd+"_f1:", f1_metric(output,label,class_num).item())
    print(sd+"auc", aucroc_metric(output,label,class_num).item())
    combined_output_list.append(output)
    combined_label_list.append(label)
combined_output = torch.cat(combined_output_list,dim=0)
combined_label = torch.cat(combined_label_list,dim=0)
print("combined_accuracy:", accuracy_metric(combined_output,combined_label,class_num).item())
print("combined_f1:", f1_metric(combined_output,combined_label,class_num).item())
print("combined_auc", aucroc_metric(combined_output,combined_label,class_num).item())

supply_accuracy: 0.34535321593284607
supply_f1: 0.34514784812927246
supplyauc 0.5908439755439758
demand_accuracy: 0.3233279585838318
demand_f1: 0.32575199007987976
demandauc 0.577078640460968
combined_accuracy: 0.33434057235717773
combined_f1: 0.3375827670097351
combined_auc 0.5839612483978271
